In [299]:
import time
import copy
import sys
from collections import OrderedDict
from collections import namedtuple
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random 
import pickle
from collections import defaultdict
import math

In [300]:
#Configure GPU (change if not M1 mac)
mps = torch.device("mps")

In [301]:
# Datasets

# Using CIFAR-10 again as in the programming assignments
# Load training data
transform_train = transforms.Compose([                                   
    transforms.RandomCrop(32, padding=4),                                       
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=True,
                                        transform=transform_train)

# Load testing data
transform_test = transforms.Compose([                                           
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True,
                                       transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [302]:
# Saving And helpers

def save_tracker(tracker, path):
  np.savetxt(path, tracker, delimiter=',') 

def save_trackers(device, filename):
  """Save all trackers and current total_time to a file."""
  torch.save((device['train_loss_tracker'], device['train_acc_tracker'], device['test_loss_tracker'], device['test_acc_tracker'], total_time), filename)
  print("Saved trackers to " + filename)

def moving_average(a, n=100):
  '''Helper function used for visualization'''
  ret = torch.cumsum(torch.Tensor(a), 0)
  ret[n:] = ret[n:] - ret[:-n]
  return ret[n - 1:] / n

# Plotting helpers! 
def make_plot(trackers, num_epochs, title, y_axis_lab, should_average=False, legend=True, fix_ax=True):
  avg_fn = moving_average if should_average else (lambda x : x) 
  x = np.arange(1, len(avg_fn(list(trackers.values())[0])) + 1)
  x = x / (len(x)/num_epochs)
  ax = plt.subplot(1,1,1)
  plt.title(title)
  plt.xlabel("Epoch")
  plt.ylabel(y_axis_lab)
  if fix_ax:
    ax.set_ylim([0, 100])
  # plt.xticks(np.arange(min(x), max(x)+1, 1))
  ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%1.0f'))

  for lab, t in trackers.items(): 
    l1, = ax.plot(x, avg_fn(t), label = lab)

  if legend:
    _ = plt.legend()
  plt.show()

def make_plot_better(trackers, num_epochs, title, y_axis_lab, should_average=False, n = 100):
  avg_fn = (lambda x : moving_average(x, n)) if should_average else (lambda x : x) 
  x = np.arange(1, len(avg_fn(list(trackers.values())[0])) + 1)
  x = x / (len(x)/num_epochs)
  ax = plt.subplot(1,1,1)
  plt.title(title)
  plt.xlabel("Epoch")
  plt.ylabel(y_axis_lab)
  # plt.xticks(np.arange(min(x), max(x)+1, 1))
  ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%1.0f'))

  for lab, t in trackers.items(): 
    l1, = ax.plot(x, avg_fn(t), label = lab)
  _ = plt.legend()
  plt.show()

In [393]:
class DatasetSplit(torch.utils.data.Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, torch.tensor(label)

def iid_sampler(dataset, num_devices, data_pct):
    '''
    dataset: PyTorch Dataset (e.g., CIFAR-10 training set)
    num_devices: integer number of devices to create subsets for
    data_pct: percentalge of training samples to give each device
              e.g., 0.1 represents 10%

    return: a dictionary of the following format:
      {
        0: [3, 65, 2233, ..., 22] // device 0 sample indexes
        1: [0, 2, 4, ..., 583] // device 1 sample indexes
        ...
      }

    iid (independent and identically distributed) means that the indexes
    should be drawn independently in a uniformly random fashion.
    '''
    total_samples = len(dataset)
    sampled = {}
    number_samples = int((data_pct)*(total_samples)) 

    for i in range(num_devices):
      sampled[i] = random.sample(range(total_samples), number_samples)
        
    return sampled

In [394]:
# Neural net definitions

# Same ConvNet as in Assignment 2 and 3
def conv_block(in_channels, out_channels, kernel_size=3, stride=1,
               padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding,
                  bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
        )

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            conv_block(3, 32),
            conv_block(32, 32),
            conv_block(32, 64, stride=2),
            conv_block(64, 64),
            conv_block(64, 64),
            conv_block(64, 128, stride=2),
            conv_block(128, 128),
            conv_block(128, 256),
            conv_block(256, 256),
            nn.AdaptiveAvgPool2d(1)
            )

        self.classifier = nn.Linear(256, 10)

    def forward(self, x):
        h = self.model(x)
        B, C, _, _ = h.shape
        h = h.view(B, C)
        return self.classifier(h)

In [403]:
# Federated learning helpers

# The baseline `average' function. 
def average_weights(devices,*args, **kwargs):
    '''
    devices: a list of devices generated by create_devices
    Returns an the average of the weights.
    '''
    state_dicts = [device['net'].state_dict() for device in devices]
#     max_magnitude = 0
    # initialize w_avg to tensors from device 0
    w_avg = copy.deepcopy(state_dicts[0])
    for k in w_avg.keys():
      w_avg[k] = w_avg[k].type(torch.float32)

    # for each model param
    for k in w_avg.keys():
        # for each remaining device i, add tensor state_dicts[i][k] to w_avg[k]
        for i in range(1, len(devices)):
#             max_magnitude = max(max_magnitude, abs(torch.max(state_dicts[i][k].type(torch.float32))))
            w_avg[k] += (state_dicts[i][k].type(torch.float32))
        # compute average
        w_avg[k] /= float(len(devices))
    return w_avg


def get_devices_for_round(devices, device_pct):  
    return random.sample(devices, int(device_pct * len(devices)))

def create_device(net, device_id, trainset, data_idxs, lr=0.1,
                  milestones=None, batch_size=128):
    if milestones == None:
        milestones = [25, 50, 75]

    device_net = copy.deepcopy(net)
    optimizer = torch.optim.SGD(device_net.parameters(), lr=lr, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                     milestones=milestones,
                                                     gamma=0.1)
    device_trainset = DatasetSplit(trainset, data_idxs)
    device_trainloader = torch.utils.data.DataLoader(device_trainset,
                                                     batch_size=batch_size,
                                                     shuffle=True)
    return {
        'net': device_net,
        'id': device_id,
        'dataloader': device_trainloader, 
        'optimizer': optimizer,
        'scheduler': scheduler,
        'train_loss_tracker': [],
        'train_acc_tracker': [],
        'test_loss_tracker': [],
        'test_acc_tracker': [],
        }

In [404]:
# Local device training and testing
def train(epoch, device, criterion):
    device['net'].train()
    train_loss, correct, total = 0, 0, 0
    for batch_idx, (inputs, targets) in enumerate(device['dataloader']):
        inputs, targets = inputs.to(mps), targets.to(mps)
        device['optimizer'].zero_grad()
        outputs = device['net'](inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        device['optimizer'].step()
        train_loss += loss.item()
        device['train_loss_tracker'].append(loss.item())
        loss = train_loss / (batch_idx + 1)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100. * correct / total
        dev_id = device['id']
        sys.stdout.write(f'\r(Device {dev_id}/Epoch {epoch}) ' + 
                         f'Train Loss: {loss:.3f} | Train Acc: {acc:.3f}')
        sys.stdout.flush()
    device['train_acc_tracker'].append(acc)
    sys.stdout.flush()

def test(epoch, device, criterion, testloader = testloader):
    device['net'].eval()
    test_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(mps), targets.to(mps)
            outputs = device['net'](inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            device['test_loss_tracker'].append(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            loss = test_loss / (batch_idx + 1)
            acc = 100.* correct / total
    sys.stdout.write(f' | Test Loss: {loss:.3f} | Test Acc: {acc:.3f}\n')
    sys.stdout.flush()  
    acc = 100.*correct/total
    device['test_acc_tracker'].append(acc)



In [407]:
# Aggregation function tests (the main experiment routine)

# Given two different sets of aggregated weights, 
# gives a value representing the difference between them,
# as a way to measure the direct cost of using our aggregated function
def diff_aggregated_weights(strat, baseline):
    result = 0 
    for k in strat.keys():
        result += torch.linalg.norm(strat[k] - baseline[k])
    return result

# A data type for experiment results
BackdoorResult = namedtuple("BackdoorResult", ["scheme_loss", "test_accuracy", "backdoor_success", "devices", "avg_weight_history"])

# The main routine! 
def run_federated_test(agg_fn = average_weights,  # Pass in aggregation function, 
                                                  # Device list -> aggregated weights 
                       rounds = 10,               # Rounds of FL
                       local_epochs = 4,          # Epochs per device                      
                       num_devices = 50,          # Total # devices
                       device_pct = 0.1,          # % of devices per round
                       data_pct = 0.1,            # % of data each device gets
                       net = ConvNet().to(mps),   # Net design; make sure on mps
                       evil_round = None,         # If None, no attack; else attacker will mount attack on this round
                       attacker_strategy = None,  # device --> void --- set up the local weights on the attacker
                       evil_device_id = None,     # Which device attacks? 
                       evaluate_attack = None,    # Function devices --> <any> measuring how well the attack worked at the end
                       output_filename = None, 
                       snapshot = True, 
                       resume_from_snap = None):   
    def lighten_device(d):
        return {
            k: d[k] for k in ( 'id', 'train_loss_tracker', 'train_acc_tracker', 'test_loss_tracker', 'test_acc_tracker')
        }                                   
    
    criterion = nn.CrossEntropyLoss()
    data_idxs = iid_sampler(trainset, num_devices, data_pct)

    devices = [create_device(net, i, trainset, data_idxs[i]) for i in range(num_devices)]
    
    scheme_loss = []
#     max_magnitudes = []
    avg_weight_history = []
    _starting_round_num = 0
    
    if resume_from_snap: 
        print("Resuming from snapshot!\n")
        # Load what we can
        result = resume_from_snap
        scheme_loss = result.scheme_loss
        avg_weight_history = result.avg_weight_history
        partial_devices = result.devices

        # Fresh data and devices
        criterion = nn.CrossEntropyLoss()
        data_idxs = iid_sampler(trainset, num_devices, data_pct)

        devices = [create_device(net, i, trainset, data_idxs[i]) for i in range(num_devices)]

        # Restore the devices with the info from the trackers
        def restore_device(old_device, new_device):
            for k in ('id', 'train_loss_tracker', 'train_acc_tracker', 'test_loss_tracker', 'test_acc_tracker'):
                new_device[k] = old_device[k]
            return new_device
        
        print("Restoring devices\n")
        devices = [restore_device(partial_devices[i], devices[i]) for i in range(len(devices))]
        for device in devices:
            device['net'].load_state_dict(avg_weight_history[-1])
        
        _starting_round_num = len(result.test_accuracy)
        print("Finished restoring\n")


        
    ## IID Federated Learning
    start_time = time.time()
    for round_num in range(_starting_round_num, rounds):
        round_start_time = time.time()
        # Part 1.3: Implement getting devices for each round here
        round_devices = get_devices_for_round(devices, device_pct)

        print('Round: ', round_num)
        # Train locally 
        for device in round_devices:
            for local_epoch in range(local_epochs):
                train(local_epoch, device, criterion)
        
        # One device becomes evil if required
        if (evil_round and round_num == evil_round):
            assert (evil_device_id is not None)
            assert (attacker_strategy is not None)
            print("Attacking!\n")
            
            attacker_strategy(devices[evil_device_id])
            # Make sure evil guy gets averaged in 
            if evil_device_id not in round_devices:
                round_devices.append(devices[evil_device_id])
            print("Finished attacking\n")
            
        
        # Weight averaging
        w_baseline = average_weights(round_devices)
        w_avg = agg_fn(round_devices, avg_weight_history[-1] if avg_weight_history != [] else None)
#         max_magnitudes.append(max_magnitude)
        
        # Track the difference between the two; should be 0 if straight average
        scheme_loss.append((float(diff_aggregated_weights(w_avg, w_baseline))))
        
        avg_weight_history.append(copy.deepcopy(w_avg))
        
        # Gradients         
        for device in devices:
            device['net'].load_state_dict(w_avg)
            device['optimizer'].zero_grad()
            device['optimizer'].step()
            device['scheduler'].step()

        # test accuracy after aggregation
        # device 0 is the unique device with all of the 
        # test accuracies and losses in its tracker
        test(round_num, devices[0], criterion)
        
        print(f"\nDiff: {scheme_loss[-1]}\n")
        print(f"Round time: {time.time() - start_time} \n")
        if snapshot:
            intermediate_result = BackdoorResult(
                scheme_loss = scheme_loss, 
                test_accuracy = devices[0]["test_acc_tracker"], 
                backdoor_success = evaluate_attack(devices) if evaluate_attack is not None else None, 
                devices = [lighten_device(d) for d in devices], 
                avg_weight_history = avg_weight_history
            )


            if output_filename is not None: 
                print("Writing snapshot\n")
                with open(f"snapshot_{output_filename}", 'wb') as file: 
                    pickle.dump(intermediate_result, file)
        
    total_time = time.time() - start_time
    print('Total training time: {} seconds'.format(total_time))
    
    # Pack up everything we care about and the devices for good measure
    result = BackdoorResult(
        scheme_loss = scheme_loss, 
        test_accuracy = devices[0]["test_acc_tracker"], 
        backdoor_success = evaluate_attack(devices) if evaluate_attack is not None else None, 
        devices = [lighten_device(d) for d in devices], 
        avg_weight_history = avg_weight_history
    )

    
    if output_filename is not None: 
        with open(output_filename, 'wb') as file: 
            print("Writing file\n")
            pickle.dump(result, file)
    
    return result


# Load output files back into memory
def load_result(filename):
    with open(filename, 'rb') as file: 
        return pickle.load(file)

In [408]:
# Example usage
results = run_federated_test(local_epochs=1, num_devices = 10, rounds = 2, output_filename = "testout.pickle")

Round:  0
(Device 5/Epoch 0) Train Loss: 2.025 | Train Acc: 23.580 | Test Loss: 2.479 | Test Acc: 22.110

Diff: 0.0

Round time: 17.671808004379272 

Writing snapshot

Round:  1
(Device 8/Epoch 0) Train Loss: 1.886 | Train Acc: 28.580 | Test Loss: 2.093 | Test Acc: 26.130

Diff: 0.0

Round time: 35.062156200408936 

Writing snapshot

Total training time: 35.09929609298706 seconds
Writing file



In [409]:
# # Can also load it back
# results = load_result("snapshot_testout.pickle")
# results

In [309]:
# A silly attack that just sends massive weights all of magnitude 10000
def sample_attack(device):
    '''
    devices: a list of devices generated by create_devices
    Returns an the average of the weights.
    '''
    weights = device["net"].state_dict().copy()
    
    for w in weights.keys():
        weights[w] = torch.full(weights[w].size(), 10000)
    
    device['net'].load_state_dict(weights)

In [310]:
# Aggregation function that squashes any weights with magnitude over 1000 (except on the first device but whatever)
def super_smart_aggregation(devices):
    '''
    devices: a list of devices generated by create_devices
    Returns an the average of the weights, excluding huge updates.
    '''
    state_dicts = [device['net'].state_dict() for device in devices]
    # initialize w_avg to tensors from device 0
    w_avg = copy.deepcopy(state_dicts[0])
    for k in w_avg.keys():
        w_avg[k] = w_avg[k].type(torch.float32)

    # for each model param
    for k in w_avg.keys():
        # for each remaining device i, add tensor state_dicts[i][k] to w_avg[k]
        for i in range(1, len(devices)):
            if (torch.max(state_dicts[i][k].type(torch.float32)) <= 1000): 
                w_avg[k] += (state_dicts[i][k].type(torch.float32))
        # compute average
        w_avg[k] /= float(len(devices))
    return w_avg

In [311]:
# Example of how this works

# Here, we carry out the attack but use the ordinary average
results_straight_avg = run_federated_test(agg_fn = (lambda x : average_weights(x)[0]),                    
                                         rounds = 3,              
                                         local_epochs = 1,                             
                                         num_devices = 10,         
                                         device_pct = 0.2,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evil_round = 1,        
                                         attacker_strategy = sample_attack,  
                                         evil_device_id = 2,     
                                         evaluate_attack = None, 
                                         output_filename = "baseline_trivial_attack.pickle")  
# Observe that it absolutely destroys our model

Round:  0
(Device 9/Epoch 0) Train Loss: 2.168 | Train Acc: 18.415

KeyboardInterrupt: 

In [92]:
# Now we do it again but with the aggregation that rejects huge updates
results_reject_huge = run_federated_test(agg_fn = super_smart_aggregation,                    
                                         rounds = 3,              
                                         local_epochs = 1,                             
                                         num_devices = 10,         
                                         device_pct = 0.2,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evil_round = 1,        
                                         attacker_strategy = sample_attack,  
                                         evil_device_id = 2,     
                                         evaluate_attack = None,
                                         output_filename = "simple_attack_and_defense.pickle")  

Round:  0
(Device 7/Epoch 0) Train Loss: 2.018 | Train Acc: 23.180 | Test Loss: 18.179 | Test Acc: 12.360

Diff: 84.85118865966797

Round:  1
(Device 9/Epoch 0) Train Loss: 1.873 | Train Acc: 29.680Attacking!

 | Test Loss: 3.511 | Test Acc: 17.340

Diff: 10114592.0

Round:  2
(Device 2/Epoch 0) Train Loss: 1.789 | Train Acc: 32.900 | Test Loss: 2.722 | Test Acc: 23.900

Diff: 46.39040756225586

Total training time: 56.98331904411316 seconds


In [ ]:
# Here, we compute a real baseline over 100 epochs and typical averaging. Good starting point for future 
# tests; can resume from this snapshot.
baseline = run_federated_test(agg_fn = (lambda x : average_weights(x)[0]),                    
                                         rounds = 100,              
                                         local_epochs = 4,                             
                                         num_devices = 50,         
                                         device_pct = 0.2,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evaluate_attack = None, 
                                         output_filename = "baseline.pickle")  

Round:  0
(Device 2/Epoch 3) Train Loss: 1.727 | Train Acc: 34.8200 | Test Loss: 2.821 | Test Acc: 10.200

Diff: 0.0

Round:  1
(Device 24/Epoch 3) Train Loss: 1.562 | Train Acc: 41.420 | Test Loss: 1.498 | Test Acc: 44.420

Diff: 0.0

Round:  2
(Device 21/Epoch 3) Train Loss: 1.314 | Train Acc: 50.700 | Test Loss: 1.242 | Test Acc: 54.680

Diff: 0.0

Round:  3
(Device 9/Epoch 3) Train Loss: 1.142 | Train Acc: 59.3200 | Test Loss: 1.086 | Test Acc: 60.890

Diff: 0.0

Round:  4
(Device 15/Epoch 3) Train Loss: 1.108 | Train Acc: 59.800 | Test Loss: 0.965 | Test Acc: 65.690

Diff: 0.0

Round:  5
(Device 1/Epoch 3) Train Loss: 1.012 | Train Acc: 63.1400 | Test Loss: 0.869 | Test Acc: 69.140

Diff: 0.0

Round:  6
(Device 28/Epoch 3) Train Loss: 0.883 | Train Acc: 68.640 | Test Loss: 0.815 | Test Acc: 71.450

Diff: 0.0

Round:  7
(Device 10/Epoch 3) Train Loss: 0.804 | Train Acc: 72.000 | Test Loss: 0.757 | Test Acc: 73.380

Diff: 0.0

Round:  8
(Device 41/Epoch 3) Train Loss: 0.766 | Train 

(Device 11/Epoch 3) Train Loss: 0.174 | Train Acc: 94.300 | Test Loss: 0.386 | Test Acc: 88.120

Diff: 0.0

Round:  70
(Device 0/Epoch 3) Train Loss: 0.190 | Train Acc: 94.0800 | Test Loss: 0.386 | Test Acc: 88.090

Diff: 0.0

Round:  71
(Device 31/Epoch 3) Train Loss: 0.166 | Train Acc: 94.200 | Test Loss: 0.386 | Test Acc: 88.120

Diff: 0.0

Round:  72
(Device 1/Epoch 1) Train Loss: 0.164 | Train Acc: 94.3820

In [ ]:
baseline = load_result("baseline.pickle")

In [312]:
# For example, now can get things like
# baseline.test_accuracy

In [45]:
# We can resume from a snapshot:
# NOTE : keep all the basic params (local epochs, num devices, net, etc) the same
# when resuming from a snapshot 
baseline2 = run_federated_test(agg_fn = (lambda x : average_weights(x)[0]),                    
                                         rounds = 101,              
                                         local_epochs = 4,                             
                                         num_devices = 50,         
                                         device_pct = 0.2,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evaluate_attack = None, 
                                         output_filename = "baseline2.pickle", 
                                         resume_from_snap = baseline )  # we pass in a results object here

Resuming from snapshot!

Restoring devices

Finished restoring

Round:  100
(Device 27/Epoch 3) Train Loss: 0.324 | Train Acc: 88.320 | Test Loss: 0.409 | Test Acc: 86.340

Diff: 0.0

Round time: 664.8843719959259 

Writing snapshot

Total training time: 667.9921019077301 seconds
Writing file



In [41]:
baseline2_loaded = load_result("baseline2.pickle")
baseline2

In [374]:
# Now we can move on to a real defense. 

# We create factory functions that let us parametrize a double sigmoid using threshholds

def torch_scaled_sigmoid_factory(a, c = 0.005, left = False):
    
    def f(x):
        scaled = torch.sub(x, a)

        divisor = c if left else -1*c

        scaled = torch.div(scaled, divisor)

        return torch.sigmoid(scaled)
    return f

def torch_double_sigmoid_factory(a,b,c = 0.005):
    left = torch_scaled_sigmoid_factory(a, left = True)
    right = torch_scaled_sigmoid_factory(b, left = False) 
    def f(x):
        return (left(x)*right(x))
    return f

my_sigmoid = torch_double_sigmoid_factory(-1,1)

In [375]:
# Make sure it works
# torch_double_sigmoid_factory(-2,2)(torch.Tensor([-5,-4,-3,-2,-1.5,-1.4,1.3,-1,0,1,1.5,1.6,2,3,4]))

In [414]:
# Creates a sigmoid defense,
#     keys_to_range_mapping maps each layer name in the state dict to the sigmoid bounds for processing it
#     All other layers just get identity function-ed
watcher = {}
def make_sigmoid_defense(keys_to_range_mapping, stickiness = 0):
    
    sigmoid_dict = defaultdict(lambda : (lambda x: 1))
    
    for k,(a,b) in keys_to_range_mapping.items():
        sigmoid_dict[k] = torch_double_sigmoid_factory(a,b)
    
    def f(devices, previous_round_weights):
        '''
        devices: a list of devices generated by create_devices
        Returns an the average of the weights preprocessed by sigmoid
        '''
        state_dicts = [device['net'].state_dict() for device in devices]
        # initialize w_avg to tensors from device 0
        w_avg = copy.deepcopy(state_dicts[0])
        for k in w_avg.keys():
            sig = sigmoid_dict[k]
            x = w_avg[k].type(torch.float32)
            prev_round_x = previous_round_weights[k].type(torch.float32) if  previous_round_weights is not None else x
            print("Diff\n")
            watcher[k] = (x - prev_round_x)
            w_avg[k] = x*sig(x - prev_round_x)
            for i in range(stickiness):
                w_avg[k] += x*sig(x - prev_round_x)                

        # for each model param
        for k in w_avg.keys():
            sig = sigmoid_dict[k]
            # for each remaining device i, add tensor state_dicts[i][k] to w_avg[k]
            prev_round_x = previous_round_weights[k].type(torch.float32)  if  previous_round_weights is not None else x
            for i in range(1, len(devices)):
                x = (state_dicts[i][k].type(torch.float32))
                w_avg[k] += x*sig(x - prev_round_x)
            # compute average
            w_avg[k] /= float(len(devices) + stickiness)
        return w_avg
    return f

In [415]:
# These are the actual layer weights that we care about
[x for x in baseline.avg_weight_history[-1].keys() if '.0.weight' in x]

['model.0.0.weight',
 'model.1.0.weight',
 'model.2.0.weight',
 'model.3.0.weight',
 'model.4.0.weight',
 'model.5.0.weight',
 'model.6.0.weight',
 'model.7.0.weight',
 'model.8.0.weight']

In [416]:
# Now lets set up a potential sigmoid defense
empirical_cutoffs = {
    'model.0.0.weight' : (-2,2),
    'model.1.0.weight' : (-2,2),
    'model.2.0.weight' : (-.7,.7),
    'model.3.0.weight' : (-.6,.6),
    'model.4.0.weight' : (-0.7,0.7),
    'model.5.0.weight' : (-0.5,0.5),
    'model.6.0.weight' : (-0.3,0.3),
    'model.7.0.weight' : (-0.2,0.2),
    'model.8.0.weight' : (-0.2,0.2),
}
sigmoid_aggregation = make_sigmoid_defense(empirical_cutoffs, stickiness=25)

In [318]:
# sigmoid_resumption = run_federated_test(agg_fn = sigmoid_accuracy,                    
#                                          rounds = 110,              
#                                          local_epochs = 1,                             
#                                          num_devices = 50,         
#                                          device_pct = 0.05,          
#                                          data_pct = 0.1,           
#                                          net = ConvNet().to(mps),  
#                                          evaluate_attack = None, 
#                                          output_filename = None,
#                                          snapshot = False,
#                                          resume_from_snap = baseline )  # we pass in a results object here

In [389]:
# Now lets make an attack
# Makes an attack that puts noise in all of target layers. 
# Noise is uniform over a,b
def noise_attack_factory(target_layers, a,b):
    def attack(device):
        '''
        devices: a list of devices generated by create_devices
        Returns an the average of the weights.
        '''
        weights = device["net"].state_dict().copy()

        for w in weights.keys():
            if w in target_layers:
                weights[w] = (a - b) * torch.rand(weights[w].size()) + b

        device['net'].load_state_dict(weights)
    return attack

In [390]:
# We attack these layers with weights ranging from -4,4
target_layers = ['model.0.0.weight',
 'model.1.0.weight',
 'model.2.0.weight',
 'model.3.0.weight',
 'model.4.0.weight',
 'model.5.0.weight',
 'model.6.0.weight',
 'model.7.0.weight',
 'model.8.0.weight']
big_noise_attack = noise_attack_factory(target_layers,-3,3)

In [391]:
# We'll resume from our baseline model
baseline = load_result("baseline.pickle")

In [279]:
sigmoid_against_noise_attack = run_federated_test(agg_fn = sigmoid_aggregation,                    
                                         rounds = 130, # go 30 pounds past where the baseline left off              
                                         local_epochs = 4,        # all else the same                     
                                         num_devices = 50,         
                                         device_pct = 0.2,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evil_round = 110, # attack after 10 rounds more training with the new acc fn        
                                         attacker_strategy = big_noise_attack, # device 2 will carry out big noise attack  
                                         evil_device_id = 2,     
                                         evaluate_attack = None, # we will evaluate manually afterwards 
                                         output_filename = "sigmoid_against_noise_attack_1.pickle",
                                         resume_from_snap = baseline, #pick up where baseline left off  
                                         snapshot = True ) # 

Resuming from snapshot!

Restoring devices

Finished restoring

Round:  100
(Device 43/Epoch 3) Train Loss: 0.293 | Train Acc: 89.860 | Test Loss: 0.589 | Test Acc: 82.010

Diff: 87.77852630615234

Round time: 668.7943501472473 

Writing snapshot

Round:  101
(Device 9/Epoch 3) Train Loss: 0.320 | Train Acc: 88.5200 | Test Loss: 0.493 | Test Acc: 84.380

Diff: 70.42446899414062

Round time: 1265.7698509693146 

Writing snapshot

Round:  102
(Device 40/Epoch 3) Train Loss: 0.216 | Train Acc: 92.800 | Test Loss: 0.565 | Test Acc: 83.180

Diff: 77.08517456054688

Round time: 1843.9197390079498 

Writing snapshot

Round:  103
(Device 43/Epoch 3) Train Loss: 0.384 | Train Acc: 86.640 | Test Loss: 0.663 | Test Acc: 79.490

Diff: 66.0071029663086

Round time: 2419.1470758914948 

Writing snapshot

Round:  104
(Device 48/Epoch 3) Train Loss: 0.181 | Train Acc: 93.620 | Test Loss: 0.593 | Test Acc: 82.430

Diff: 65.98039245605469

Round time: 2990.5921189785004 

Writing snapshot

Round:  105
(

In [417]:
baseline = load_result("baseline.pickle")
determine_cutoffs = run_federated_test(agg_fn = sigmoid_aggregation,                    
                                         rounds = 101, # go 30 pounds past where the baseline left off              
                                         local_epochs = 1,        # all else the same                     
                                         num_devices = 50,         
                                         device_pct = 0.05,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evil_round = None, # attack after 10 rounds more training with the new acc fn        
                                         attacker_strategy = None, # device 2 will carry out big noise attack  
                                         evil_device_id = None,     
                                         evaluate_attack = None, # we will evaluate manually afterwards 
                                         output_filename = None,
                                         resume_from_snap = baseline, #pick up where baseline left off  
                                         snapshot = False ) # 

Resuming from snapshot!

Restoring devices

Finished restoring

Round:  100
(Device 13/Epoch 0) Train Loss: 0.334 | Train Acc: 88.200Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

Diff

 | Test Loss: 1.250 | Test Acc: 72.800

Diff: 34.20332717895508

Round time: 34.021363258361816 

Total training time: 34.021379232406616 seconds


In [418]:
watcher

{'model.0.0.weight': tensor([[[[-9.4310e-03, -6.7759e-03,  3.0469e-03],
           [-1.7775e-02, -1.1662e-02,  1.3760e-03],
           [-1.4200e-02, -1.4083e-02, -9.8117e-03]],
 
          [[-6.7062e-03, -7.9544e-03, -1.8204e-03],
           [-1.7414e-02, -1.4476e-02, -5.0078e-03],
           [-8.6145e-03, -1.0224e-02, -6.3146e-03]],
 
          [[-5.0432e-03, -4.2537e-03, -2.3357e-03],
           [-1.1462e-02, -8.9888e-03, -8.9411e-03],
           [-3.5206e-03, -6.3042e-03, -1.3175e-02]]],
 
 
         [[[-9.8976e-03, -1.5522e-02, -1.1095e-02],
           [-9.0138e-03, -1.3783e-02, -8.9821e-03],
           [ 3.9757e-03,  1.1459e-03, -2.5883e-03]],
 
          [[ 3.0415e-03, -1.8064e-03, -2.0479e-04],
           [ 9.1547e-03,  6.0731e-03,  7.8201e-03],
           [ 2.5521e-02,  2.4583e-02,  1.8199e-02]],
 
          [[ 3.8094e-03, -7.5914e-04, -2.2959e-03],
           [ 1.3139e-02,  9.9015e-03,  5.5838e-03],
           [ 3.1718e-02,  3.0887e-02,  1.9483e-02]]],
 
 
         [[[-1.9414e

In [330]:
# def make_sigmoid_on_cutoff(a,c=0.005, left = False):
#     def f(x):
#         if(not left and x > (a + 2)):
#             return 0
#         if(left and x < (a - 2)):
#             return 0
#         exponent = (-(x-a))/(c if left else -1*c)
#         return 1/(1 + (math.e**(exponent)))
#     return f 

# def make_double_sigmoid(a,b):
#     left = make_sigmoid_on_cutoff(a, left = True)
#     right = make_sigmoid_on_cutoff(b, left = False)
#     def f(x):
#         return left(x)*right(x)
#     return f
# baseline2.devices[0]['net']

def make_test_device(trainset):
    data_idxs = iid_sampler(trainset, 1, .1)
    return create_device(ConvNet().to(mps), 0, trainset, data_idxs[0]) 



In [302]:
# 


In [303]:
# big_noise_attack(device)

In [304]:
# device['net'].state_dict()['model.0.0.weight']

In [305]:
# trainset[1]

In [289]:
# labels = [x[1] for x in trainset if x[1] == 1]

In [306]:
# len(labels)

In [311]:
def swap_classes_dataset(s, a, b):
    s = list(s)
    for i in range(len(s)):
        if s[i][1] == a:
            s[i] = s[i][0],b
        if s[i][1] == b:
            s[i] = s[i][0],a 
    
    return s

# Make switch classes attack
def switch_classes_attack(a,b, train_epochs):
    transform_train = transforms.Compose([                                   
        transforms.RandomCrop(32, padding=4),                                       
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    evil_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                            download=True,
                                            transform=transform_train)
    evil_trainset = swap_classes_dataset(evil_trainset, a,b)
    print("swapping\n")
    
    # Load testing data
    transform_test = transforms.Compose([                                           
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    
    evil_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True,
                                           transform=transform_test)
    
    evil_testset = swap_classes_dataset(evil_testset, a,b)
    
    evil_testloader = torch.utils.data.DataLoader(evil_testset, batch_size=128, shuffle=False)
    
    attack_idx = iid_sampler(evil_trainset, 1, 1)[0]
    
    
    
    evil_device_trainset = DatasetSplit(evil_trainset, attack_idx)
    evil_device_trainloader = torch.utils.data.DataLoader(evil_device_trainset,
                                                     batch_size=128,
                                                     shuffle=True)

    criterion = nn.CrossEntropyLoss()

    def attack(device):
        '''
        devices: a list of devices generated by create_devices
        Returns an the average of the weights.
        '''
        device['dataloader'] = evil_device_trainloader

        for local_epoch in range(train_epochs):
            train(local_epoch, device, criterion)
        
        print("Confirm the attack worked. (This should be high)")  
        test(0, device, criterion,testloader =  evil_testloader)


        

    return attack

sca = switch_classes_attack(0,1,1)

Files already downloaded and verified
swapping

Files already downloaded and verified


In [317]:
# Now let's make an instance of this attack that performs 25 epochs of training on the evil device with the misclassified data.
# Specifically, we switch cars and airplanes
sca2 = switch_classes_attack(0,1,25)

Files already downloaded and verified
swapping

Files already downloaded and verified


In [318]:

baseline = load_result("baseline.pickle")
switch_classes_no_defense = run_federated_test(                    
                                         rounds = 130, # go 30 pounds past where the baseline left off              
                                         local_epochs = 4,        # all else the same                     
                                         num_devices = 50,         
                                         device_pct = 0.2,          
                                         data_pct = 0.1,           
                                         net = ConvNet().to(mps),  
                                         evil_round = 110, # attack after 10 rounds more training with the new acc fn        
                                         attacker_strategy = sca2, # device 2 will carry out big noise attack  
                                         evil_device_id = 2,     
                                         evaluate_attack = None, # we will evaluate manually afterwards 
                                         output_filename = "switch_classes_no_defense.pickle",
                                         resume_from_snap = baseline, #pick up where baseline left off  
                                         snapshot = True ) # 

Resuming from snapshot!

Restoring devices

Finished restoring

Round:  100
(Device 30/Epoch 3) Train Loss: 0.448 | Train Acc: 84.700 | Test Loss: 0.389 | Test Acc: 87.390

Diff: 0.0

Round time: 218.5696198940277 

Writing snapshot

Round:  101
(Device 3/Epoch 3) Train Loss: 0.327 | Train Acc: 88.8800 | Test Loss: 0.382 | Test Acc: 87.490

Diff: 0.0

Round time: 425.6108150482178 

Writing snapshot

Round:  102
(Device 14/Epoch 3) Train Loss: 0.222 | Train Acc: 92.600 | Test Loss: 0.408 | Test Acc: 86.950

Diff: 0.0

Round time: 632.2491769790649 

Writing snapshot

Round:  103
(Device 25/Epoch 3) Train Loss: 0.335 | Train Acc: 88.440 | Test Loss: 0.419 | Test Acc: 86.930

Diff: 0.0

Round time: 830.303952217102 

Writing snapshot

Round:  104
(Device 40/Epoch 3) Train Loss: 0.231 | Train Acc: 93.040 | Test Loss: 0.383 | Test Acc: 87.780

Diff: 0.0

Round time: 1036.1185810565948 

Writing snapshot

Round:  105
(Device 42/Epoch 3) Train Loss: 0.255 | Train Acc: 91.180 | Test Loss: 0.3

In [321]:
switch_classes_no_defense = load_result("switch_classes_no_defense.pickle")

In [322]:
switch_classes_no_defense.test_accuracy

[10.2,
 44.42,
 54.68,
 60.89,
 65.69,
 69.14,
 71.45,
 73.38,
 76.61,
 77.53,
 79.7,
 77.91,
 79.95,
 81.38,
 82.4,
 82.86,
 83.43,
 83.6,
 84.71,
 85.13,
 85.46,
 85.9,
 85.79,
 86.54,
 86.48,
 87.19,
 87.33,
 87.52,
 87.52,
 87.37,
 87.48,
 87.65,
 87.62,
 87.56,
 87.58,
 87.7,
 87.63,
 87.62,
 87.8,
 87.76,
 87.92,
 87.82,
 87.87,
 87.91,
 87.96,
 87.89,
 88.11,
 87.85,
 87.89,
 88.12,
 88.09,
 87.96,
 88.01,
 87.94,
 88.05,
 88.0,
 87.96,
 88.07,
 88.05,
 88.0,
 88.17,
 88.07,
 88.23,
 88.07,
 88.08,
 87.99,
 88.1,
 88.11,
 88.13,
 88.12,
 88.09,
 88.12,
 88.08,
 88.12,
 88.13,
 88.11,
 88.06,
 88.12,
 88.07,
 88.15,
 88.1,
 88.06,
 88.16,
 88.12,
 88.14,
 88.14,
 88.18,
 88.12,
 88.11,
 88.09,
 88.07,
 88.08,
 88.16,
 88.09,
 88.23,
 88.22,
 88.13,
 88.04,
 88.13,
 88.05,
 87.39,
 87.49,
 86.95,
 86.93,
 87.78,
 87.84,
 87.54,
 88.29,
 88.46,
 88.28,
 88.34,
 88.5,
 88.47,
 88.76,
 88.92,
 89.23,
 89.64,
 88.94,
 89.04,
 89.17,
 89.55,
 89.55,
 89.59,
 89.69,
 89.05,
 90.25,
 90.

In [367]:
test_device = make_test_device(trainset)
test_device['net'].load_state_dict(switch_classes_no_defense.avg_weight_history[112])
# def test(epoch, device, criterion, testloader = testloader):

<All keys matched successfully>

In [368]:
test(0, test_device, nn.CrossEntropyLoss(), testloader)

 | Test Loss: 0.368 | Test Acc: 88.560


In [369]:
def limit_test_set(s, labels):
    s = list(s)
    s = [s for s in s if s[1] in labels]
    return s



def make_testloader_subset(labels):
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True,
                                       transform=transform_test)
    
    restricted_test_set = limit_test_set(testset, labels)
    print(len(restricted_test_set))
#     restricted_test_set = swap_classes_dataset(restricted_test_set, 0, 1)
    
    restricted_testloader = torch.utils.data.DataLoader(restricted_test_set, batch_size=128, shuffle=False)
    return restricted_testloader

In [373]:
checker_test_set = make_testloader_subset([0,1])

Files already downloaded and verified
2000


In [371]:
test(0, test_device, nn.CrossEntropyLoss(), checker_test_set)

 | Test Loss: 0.392 | Test Acc: 89.400
